In [1]:
from pathlib import Path
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd

import os 
import itertools
import re
import math
import csv
import random
from sklearn import tree
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from scipy.fftpack import fftn, ifftn, fft, ifft
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import signal
import pickle
from sklearn.externals import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import warnings

warnings.filterwarnings('ignore') 
df = pd.read_csv("noAudible/Features_25too30hz.csv")



#df = pd.read_csv("Features_all_no_audible.csv")
#df = df[(df.modeID != 'audible')]
df

C:\Users\Magomed\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


,channel_1_mav,channel_1_mav2,channel_1_rms,channel_1_iav,channel_1_ssi,channel_1_var,channel_1_wl,channel_1_iemg,channel_1_aac,channel_1_zc,...,channel_6_aac,channel_6_zc,channel_6_ssc,channel_6_wamp,channel_6_medf,channel_6_meanf,userID,sessionID,uttID,modeID
0,0.004827,0.004526,0.006212,18.859834,0.150743,0.000039,5.404816,18.859834,0.001383,357,...,0.444535,1278,2241,1068,0.000000,66.053648,1,1001,10010001,audible
1,0.004161,0.003792,0.005610,10.940351,0.082743,0.000031,3.147139,10.940351,0.001197,240,...,0.443308,794,1497,728,0.000000,57.514921,1,1001,10010002,audible
2,0.004842,0.004718,0.006667,22.918711,0.210362,0.000044,6.595381,22.918711,0.001393,434,...,0.445901,1534,2726,1316,0.000000,73.430718,1,1001,10010003,audible
3,0.004672,0.004489,0.006003,22.702916,0.175124,0.000036,6.547547,22.702916,0.001348,448,...,0.445519,1540,2792,1345,1.859504,77.039314,1,1001,10010004,audible
4,0.004638,0.004484,0.006379,25.436362,0.223146,0.000041,7.389504,25.436362,0.001347,507,...,0.446329,1716,3143,1523,7.678245,77.906715,1,1001,10010005,audible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,0.003905,0.003601,0.005662,10.640937,0.087358,0.000032,3.046365,10.640937,0.001118,245,...,0.375292,1702,2214,389,0.000000,59.203804,8,8019,80190246,whisper
4796,0.002801,0.002387,0.003902,6.223302,0.033827,0.000015,1.818702,6.223302,0.000818,211,...,0.374308,1374,1823,317,0.000000,52.306202,8,8019,80190247,whisper
4797,0.002718,0.002371,0.003678,7.145742,0.035563,0.000014,2.061635,7.145742,0.000784,242,...,0.373218,1628,2147,375,0.000000,59.650508,8,8019,80190248,whisper
4798,0.002560,0.002362,0.003602,5.112926,0.025907,0.000013,1.459807,5.112926,0.000731,182,...,0.374330,1232,1644,285,0.000000,46.464064,8,8019,80190249,whisper


In [2]:
def split_data_per_session(session_id):
    temp_df = df[(df.sessionID == session_id)]
    data = temp_df.copy()
    data = shuffle(data)
    y = data['modeID']
    data.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return train_test_split(data, y, test_size=0.2)

def split_data_session(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['sessionID'].isin(train_user_ids)]
    test_df = data.loc[data['sessionID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data_utterance(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

def split_data(train_user_ids,test_user_ids):
    temp_df = df
    data = temp_df.copy()
    train_df = data.loc[data['userID'].isin(train_user_ids)]
    test_df = data.loc[data['userID'].isin(test_user_ids)]
    X_train = train_df.copy()
    X_test = test_df.copy()
    y_train = X_train['modeID']
    y_test = X_test['modeID']
    X_train.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    X_test.drop(labels=['userID','uttID', 'sessionID','modeID'], axis=1, inplace=True)
    return X_train, X_test, y_train, y_test 

In [3]:
def lda_user1():
    train_ids = [2,3,4,5,6,7,8]
    test_ids = [1]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
def lda_user2():
    train_ids = [1,3,4,5,6,7,8]
    test_ids = [2]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user3():
    train_ids = [1,2,4,5,6,7,8]
    test_ids = [3]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 
    
def lda_user4():
    train_ids = [1,2,3,5,6,7,8]
    test_ids = [4]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score     
        
def lda_user5():
    train_ids = [1,2,3,4,6,7,8]
    test_ids = [5]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score   
    
def lda_user6():
    train_ids = [1,2,3,4,5,7,8]
    test_ids = [6]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score  
    
def lda_user7():
    train_ids = [1,2,3,4,5,6,8]
    test_ids = [7]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score    
    
def lda_user8():
    train_ids = [1,2,3,4,5,6,7]
    test_ids = [8]
    X_train, X_test, y_train, y_test = split_data(train_ids,test_ids)
    k_clf = LinearDiscriminantAnalysis()
    k_clf.fit(X_train, y_train)
    score = k_clf.score(X_test, y_test)
    print("Accuracy for session 19 is "+str(score))
    return score 

In [4]:
def lda_users():
    a = lda_user1()
    b = lda_user2()
    c = lda_user3()
    d = lda_user4()
    e = lda_user5()
    f = lda_user6()
    g = lda_user7()
    h = lda_user8()
    scores = np.array([a,b,c,d,e,f,g,h])
    print("Durchschnitt: ")
    print (scores.mean())
    print("Standartabweichung: ")
    print (scores.std())

In [5]:
lda_users()

Accuracy for session 19 is 0.34444444444444444
Accuracy for session 19 is 0.3257777777777778
Accuracy for session 19 is 0.3466666666666667
Accuracy for session 19 is 0.4166666666666667
Accuracy for session 19 is 0.4
Accuracy for session 19 is 0.41333333333333333
Accuracy for session 19 is 0.37666666666666665
Accuracy for session 19 is 0.41904761904761906
Durchschnitt: 
0.38032539682539684
Standartabweichung: 
0.034831548522665096
